In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from observation_models import Exp_UniGauss as ExpUniGauss
from observation_models import ExpGauss as ExpGauss
from observation_models import Gaussian
from sklearn.datasets.samples_generator import make_spd_matrix
from hmm_utils import normalize
from hmm import hmm_calibration

In [2]:
#hidden_states =3
sample_points = 1000
dur_lambdas =[0.5]
sigmas =[1]
prob_zeroPC =[0.3]
hidden_states = [0]*sample_points #creates an array of size 1000 with zeros
rng= np.random.RandomState(14)# sets the seed for the random number generator

In [3]:
sample_data=ExpGauss.sample_data(hidden_states, dur_lambdas, sigmas, prob_zeroPC,rng=rng)

In [4]:
taus=sample_data[0]
price_changes = sample_data[1]

In [ ]:
plt.hist(taus)
plt.title('Durations')

In [ ]:
plt.hist(price_changes)
plt.title('Price_Changes')

In [ ]:
prng = np.random.RandomState(10)
n_components = 3
n_features = 3
###This is like the set up func Unit test of hmmlearn ############
startprob = prng.rand(n_components)
startprob = startprob / startprob.sum()
transmat = prng.rand(n_components, n_components)
transmat /= np.tile(transmat.sum(axis=1)[:, np.newaxis],
                                 (1, n_components))

means = prng.randint(-20, 20, (n_components, n_features))
diag_sigmas = (1.0 + 2 * prng.rand(n_components, n_features)) ** 2 # this is used for constructing gaussian emissions            
init = 1. / n_components
priors = {
    "means": 20*means,
    #"covars":
    "pi": np.full(n_components, init),
    "tpm": np.full((n_components, n_components), init) #normalize(transmat + np.diag(prng.rand(n_components)), 1)
}

sample_size = 1000

In [ ]:
update_tag = 'tpsm'
#print transmat
states = hmm_calibration.sample_states(startprob, transmat, prng, sample_size=sample_size) #generates a sample of states
obs = Gaussian.sample_data(states, 20*means, diag_sigmas, rng=prng) #given the sample of states you create observation data
obs_model = Gaussian(priors, len(startprob), n_components) #creates an instance of the class
the_hmm = hmm_calibration(obs_model, len(startprob), priors)

In [ ]:
res = the_hmm.run_hmm(obs, update_tag=update_tag, set_up_initials_tag='uniform')

In [ ]:
print res.keys()

In [ ]:
print res['log_likelihoods'].keys()

In [ ]:
plt.plot(res['log_likelihoods']['complete'][5:])

In [ ]:
print(res['log_likelihoods']['obs_data'])

In [ ]:
print transmat #original transmat

In [ ]:
print res['tpm'][-1] #take final transmat